In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import (SystemMessagePromptTemplate, 
HumanMessagePromptTemplate, ChatPromptTemplate)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


In [9]:
load_dotenv(find_dotenv())
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

In [11]:
sentiment_analysis_prompt= """Given a user review below, classify it as a Positive or Negative review, 
Return your answer in only one Classification  word.
review: {review}
Classification: """

sentiment_template = ChatPromptTemplate.from_template(sentiment_analysis_prompt)

sentiment_chain = sentiment_template | llm | StrOutputParser()

sentiment = sentiment_chain.invoke({'review':
                          'I did not like the product'})

In [12]:
sentiment

'Negative'

In [13]:
positive_response = """You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_response)

positive_chain = positive_template | llm | StrOutputParser()

In [14]:
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'prashant@company.com'.
                Review: {review}
                Answer:"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)

negative_chain = negative_template | llm | StrOutputParser()

In [15]:
def decisionReviewer(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [ ]:
full_chain = {'sentiment': sentiment_chain, 'review': lambda x : x['review']} | RunnableLambda(decisionReviewer)

full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='Given a user review below, classify it as a Positive or Negative review, \nReturn your answer in only one Classification  word.\nreview: {review}\nClassification: '), additional_kwargs={})])
             | ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000156966CC190>, async_client=<openai.resources.chat.completions.

In [23]:
review = 'I liked the product'

final_output = full_chain.invoke({'review':review})

final_output

"Thank you so much for your positive feedback! We're thrilled to hear that you liked the product. If you have a moment, we would love for you to share your experience on social media. Your support helps us grow and reach more amazing customers like you! 🌟"